# 1. Dataset Check

In [10]:
# Libraries
import pandas as pd
from google.colab import drive
import datetime

In [11]:
# Connect to Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# read dataset
data = pd.read_csv('/content/drive/MyDrive/MasterThesis/Reddit_Data/cleaned_libra.csv')
data.head()

,thread_title,thread_time_utc,thread_score,comment_time_utc,comment_score,comment_text,subreddit,full_text
0,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 11:51:07,0,ye and the average person is miserable so what...,CryptoCurrency,degen gets fked on valentines day loses $33m i...
1,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 09:40:55,1,if he does this in the open since its technica...,CryptoCurrency,degen gets fked on valentines day loses $33m i...
2,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 09:45:38,1,presidents are now rugging the space,CryptoCurrency,degen gets fked on valentines day loses $33m i...
3,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 09:56:59,6,its not money laundering when you put absolute...,CryptoCurrency,degen gets fked on valentines day loses $33m i...
4,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 09:59:03,4,this is a classic phenomenon in the shitcoin s...,CryptoCurrency,degen gets fked on valentines day loses $33m i...


In [13]:
data.shape

(2043, 8)

In [14]:
# Droping the data aqfter 28.02.2025 to align with the blockchain dat aanalysis
data['comment_time_utc'] = pd.to_datetime(data['comment_time_utc'])
cutoff = pd.to_datetime('2025-02-28')   # no utc=True
data.drop(data[data['comment_time_utc'] > cutoff].index, inplace=True)

In [15]:
data.shape

(1963, 8)

In [16]:
data.nunique()

,0
thread_title,37
thread_time_utc,38
thread_score,33
comment_time_utc,1959
comment_score,106
comment_text,1839
subreddit,5
full_text,1878


In [17]:
# dropping duplicates in order to avoid using bots comments or removed comments from analysis
duplicates = data[data.duplicated(subset='comment_text', keep=False)]
duplicates

,thread_title,thread_time_utc,thread_score,comment_time_utc,comment_score,comment_text,subreddit,full_text
6,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 10:05:39,1,gifemotefreeemotespackjoy,CryptoCurrency,degen gets fked on valentines day loses $33m i...
89,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 22:08:53,1,removed,CryptoCurrency,degen gets fked on valentines day loses $33m i...
91,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 22:09:12,1,please be cautious with links in the the above...,CryptoCurrency,degen gets fked on valentines day loses $33m i...
95,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 12:47:23,1,removed,CryptoCurrency,degen gets fked on valentines day loses $33m i...
96,degen gets fked on valentines day loses $33m i...,2025-02-15 02:35:44,2064,2025-02-15 12:49:28,1,it appears your comment contains a url shorten...,CryptoCurrency,degen gets fked on valentines day loses $33m i...
...,...,...,...,...,...,...,...,...
1975,if you could invest in only 1 coin moving forw...,2025-02-26 15:26:59,48,2025-02-26 21:06:24,1,solana,CryptoMarkets,if you could invest in only 1 coin moving forw...
1976,if you could invest in only 1 coin moving forw...,2025-02-26 15:26:59,48,2025-02-26 21:05:52,5,pi,CryptoMarkets,if you could invest in only 1 coin moving forw...
1981,if you could invest in only 1 coin moving forw...,2025-02-26 15:26:59,48,2025-02-26 21:48:29,3,xrp,CryptoMarkets,if you could invest in only 1 coin moving forw...
1982,if you could invest in only 1 coin moving forw...,2025-02-26 15:26:59,48,2025-02-26 21:50:55,1,kaspa,CryptoMarkets,if you could invest in only 1 coin moving forw...


In [18]:
data = data.drop_duplicates(subset='comment_text', keep='first')


In [19]:
data.shape

(1840, 8)

In [20]:
data = data[data["comment_text"].str.split().str.len() > 4]

In [21]:
data.shape

(1476, 8)

# 2. Topic Modelling

In [22]:
!pip install bertopic[visualization] umap-learn hdbscan


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [23]:

# Import libraries
import pandas as pd
import umap
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
import nltk
import re
from nltk.corpus import stopwords
import spacy

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

# Text cleaning + lemmatization function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www.\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s$]", "", text)
    text = re.sub(r"\s+", " ", text).strip()

    # Lemmatize and remove stopwords
    doc = nlp(text)
    words = [token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct and not token.is_space]

    cleaned_text = " ".join(words)
    return cleaned_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
# Load cleaned dataset from previous part
df = data

# Cleaning using the prepared function
df["clean_comment_text"] = df["comment_text"].dropna().apply(clean_text)
texts = df["clean_comment_text"].dropna().tolist()

# HDBSCAN
hdbscan_model = HDBSCAN(
    min_cluster_size=10,
    min_samples=1,
    prediction_data=True
)

# UMAP model
umap_model = umap.UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)

# Embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Create BERTopic model
topic_model = BERTopic(
    hdbscan_model=hdbscan_model,
    umap_model=umap_model,
    embedding_model=embedding_model,
    calculate_probabilities=True,
    language="english",
    verbose=True
)

topics, probs = topic_model.fit_transform(texts)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-05-07 09:42:34,023 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

2025-05-07 09:42:58,841 - BERTopic - Embedding - Completed ✓
2025-05-07 09:42:58,844 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-07 09:43:13,836 - BERTopic - Dimensionality - Completed ✓
2025-05-07 09:43:13,838 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-07 09:43:14,103 - BERTopic - Cluster - Completed ✓
2025-05-07 09:43:14,114 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-07 09:43:14,224 - BERTopic - Representation - Completed ✓


In [33]:
# After checking different options, we needed to reduce the number of topics. Based on the topic visualiations numer 8 seemed the most optimal
topic_model.reduce_topics(texts, nr_topics=8)

2025-05-07 09:43:56,729 - BERTopic - Topic reduction - Reducing number of topics
2025-05-07 09:43:56,743 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-07 09:43:56,816 - BERTopic - Representation - Completed ✓
2025-05-07 09:43:56,819 - BERTopic - Topic reduction - Reduced number of topics from 9 to 8


# 3. Visualization and model evaluation

In [34]:
topic_model.visualize_topics()


In [36]:
# Visualize hierarchy of topics
topic_model.visualize_hierarchy()


In [37]:
# Visualize the top topics
topic_model.visualize_barchart(top_n_topics=10)


In [39]:
libra_topic = topic_model.get_topic_info()
libra_topic

,Topic,Count,Name,Representation,Representative_Docs
0,-1,412,-1_coin_people_buy_go,"[coin, people, buy, go, get, know, like, say, ...",[need find casino win back money lose crypto p...
1,0,664,0_coin_people_crypto_meme,"[coin, people, crypto, meme, scam, buy, like, ...",[I sure tell people buy coin make money go dum...
2,1,163,1_guy_ada_people_interview,"[guy, ada, people, interview, know, fuck, give...",[huge ada fan tell I I follow project cardano ...
3,2,132,2_trump_milei_coin_melania,"[trump, milei, coin, melania, people, would, s...",[team launch $ trump davis say people involve ...
4,3,52,3_rug_pull_rugge_rugpull,"[rug, pull, rugge, rugpull, get, rugger, good,...",[yeah point try get rug pull like solana dexs9...
5,4,32,4_xrp_hbar_sui_link,"[xrp, hbar, sui, link, solana, qnt, xlm, xdc, ...","[qnt xrp hbar I know 3 I choose order, xrp I l..."
6,5,11,5_accident_jail_traffic_prison,"[accident, jail, traffic, prison, bar, right, ...",[clown jail different ftx absolute snake behin...
7,6,10,6_day_single_valentine_hot,"[day, single, valentine, hot, wild, degen, tod...",[hot single close house sometime mile away els...


In [40]:
libra_topic.to_csv("/content/drive/MyDrive/MasterThesis/libra_topic.csv", index=False)
